### Loading Data

In [2]:
%reset -f
import random
import sys
import mne
import matplotlib.pyplot as plt
import numpy as np
import os.path as op
import os
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from scipy.spatial.distance import pdist, squareform
from auxiliary_analysis import *
from scipy.spatial.distance import squareform
mne.__version__
mne.set_log_level("WARNING")

infolder, outfolder = find_folder()
conf = dict()

# EEG parameters
resample = 200

### Imagery Preprocessing

In [69]:
# Defining triggers
trigs=list(range(31, 36))
event_ids={str(x):x for x in trigs}
all_accuracies = pd.DataFrame() # DF to store SVM results

# Preprocessing Params
start = 2 # first participant
end = 3 # last participant
random_participants = False
n_participants = 5
verbose = False

# If you want to pick a sample of random participants, change random_participants to True
# and n_participants to the sample size
if random_participants:
   participants = np.random.randint(start, end, n_participants)
else:
  participants = range(start, end)

participants = [2,3,4,6,12,18]

for i in participants:
  count = 0

  # Generate bdf file names
  id = str(i).zfill(2)
  file_name_1 = 'IR_' + id + '_S01.bdf'
  file_name_2 = 'IR_' + id + '_S02.bdf'
  file_name_3= 'IR_' + id + '_S03.bdf'

  # Loading participant's first session
  fname = op.join(infolder,file_name_1)

  # Convert to epochs
  epochs_a = preprocessing_eeg(fname, event_ids, segment_times=(-1,5), resample = resample)

  # Loading participant's second session
  fname = op.join(infolder,file_name_2)

  # Convert to epochs
  epochs_b = preprocessing_eeg(fname, event_ids, segment_times=(-1,5), resample = resample)

  # If second participant, add another session because P2 has 3 sessions
  if (i == 2):
     epochs_c = preprocessing_eeg(fname, event_ids, segment_times=(-1,5), resample = resample)
     epochs_im = mne.concatenate_epochs([epochs_a,epochs_b, epochs_c])
  else:
    fname = op.join(infolder,file_name_3)
    epochs_im = mne.concatenate_epochs([epochs_a,epochs_b])

  %reset_selective -f 4_

  # Dropping status channel
  epochs_im.drop_channels('Status')

  # Equalizing number of cases between conditions
  epochs_im.equalize_event_counts(event_ids=event_ids, method='mintime') 

  # z-scoring, thresholding, rereferencing
  epochs_im = zscore_threshold_epochs(epochs_im, 2, 3, time = True) 

  epochs_im.set_eeg_reference("average")

  # save as fif
  fname = op.join(outfolder,'S' + id + '_imag-epo.fif')
  epochs_im.save(fname, overwrite = True)

  # averaging by block
  epochs_im = block_average(epochs_im, 8)

  # save as fif again
  fname = op.join(outfolder,'S' + id + '_imag_aver-epo.fif')
  epochs_im.save(fname, overwrite = True)

  print(f'Session {i} preprocessed') if verbose else None

  # SVM setup
  im_names = ['mcy','sgo','sjo','est','tsw']
  times=[(-1, 0),(0,5),(0,4),(0,3),(0,2),(0,1),(1,2),(1,3),(1,4),(1,5),(2,3),(2,4),(2,5),(3,4),(3,5),(4,5)]
  results=[]
  durs=[];

  # Running SVM
  for j in times:
    X, Y = convert_epochs_to_2D_array(epochs_im, times=j)
    confusion, duration = run_eeg_svm(X,Y,6)
    results.append(confusion)
    durs.append(duration)

  print(f'Session {i} analysed:') if verbose else None

  accuracies = []

  for k in results:
      print(f'Accuracy of the entire set is {np.mean(squareform(k))*100:.2f}%') if verbose else None
      accuracies.append(round(np.mean(squareform(k))*100, 2))
  all_accuracies.insert(count, f'Participant {i}', accuracies, True)
  confusion_df = pd.DataFrame(results[3], columns = im_names, index = im_names)
  fname = op.join(outfolder,'conf_IR_' + str(i).zfill(2) + '_imag.csv')
  confusion_df.to_csv(fname, index = True, header = True)
  count+=1

all_accuracies.to_csv("image_accuracies2.csv", index=True)

### Perception Preprocessing

In [ ]:
# Defining triggers
trigs=list(range(101, 151))+list(range(201, 251))
event_ids={str(x):x for x in trigs}

start = 3
end = 4

# Generate an array of 4 random integers in the specified range

for i in range(start, end):
  file_name_1 = 'IR_' + str(i).zfill(2) + '_S01.bdf'
  file_name_2 = 'IR_' + str(i).zfill(2) + '_S02.bdf'

  # Loading first session
  fname = op.join(infolder,file_name_1)

  # Load data
  epochs_a = preprocessing_eeg(fname, event_ids, segment_times=(-0.1,1), resample = resample)
  epochs_a = epochs_a[100:]

  # Loading second session
  file_name=file_name_2
  fname = op.join(infolder,file_name_2)

  # Load data
  epochs_b = preprocessing_eeg(fname, event_ids, segment_times=(-0.1,1), resample = resample)
  epochs_b = epochs_b[100:]

  # creating one data
  epochs_perc = mne.concatenate_epochs([epochs_a,epochs_b])
  %reset_selective -f 4_

  #dropping status channel
  epochs_perc.drop_channels('Status')

  # equalizing number of cases between conditions
  epochs_perc.equalize_event_counts(event_ids=event_ids, method='mintime') 

  # adding metadata
  epochs_perc.metadata = create_meta_events_perception(epochs_perc.events[:,2]) 


  # zscoring, thresholding, rereferencing
  epochs_perc = zscore_threshold_epochs(epochs_perc, dims=2, threshold=3) 
  epochs_perc.set_eeg_reference("average")
  
  # saving
  fname = op.join(outfolder,'S' + str(i).zfill(2) + '_perc-epo.fif')
  epochs_perc.save(fname, overwrite = True)

  # averaging by block
  epochs_perc = block_average(epochs_perc, 4)

  # saving
  fname = op.join(outfolder,'S' + str(i).zfill(2) + '_perc_aver-epo.fif')
  epochs_perc.save(fname, overwrite = True)

  os.system(f'say "Participant {i} preprocessed"')

  times=[(0.05,0.65)]
  results=[]
  durs=[]

  for j in times:
    X, Y = convert_epochs_to_2D_array(epochs_perc, times=j)
    confusion, duration = run_eeg_svm(X, Y, 12)
    results.append(confusion)
    durs.append(duration)

  # os.system(f'say "Session {i} analysed"')

  confusion=results[0]
  print(f'Session {i} analysed with dimensions {d}:')
  print(f'Accuracy of the upright stimuli is {np.mean(squareform(confusion[0:50,0:50]))*100:.4f}%')
  print(f'Accuracy of the upright unfamiliary stimuli is {np.mean(squareform(confusion[0:25,0:25]))*100:.4f}%')
  print(f'Accuracy of the upright famous stimuli is {np.mean(squareform(confusion[25:50,25:50]))*100:.4f}%')
  print(f'Accuracy of the inverted stimuli is {np.mean(squareform(confusion[50:,50:]))*100:.4f}%')
  print(f'Accuracy of the inverted unfamiliar stimuli is {np.mean(squareform(confusion[50:75,50:75]))*100:.4f}%')
  print(f'Accuracy of the inverted famous stimuli is {np.mean(squareform(confusion[75:,75:]))*100:.4f}%')
  print(f'Accuracy of the entire set is {np.mean(squareform(confusion))*100:.4f}%')

  ## Correlation results
  corr=np.corrcoef(squareform(confusion[0:25,0:25]),squareform(confusion[50:75,50:75]))
  print(f'Correlation between identity discrimination in upright and inverted famous faces is {corr[0,1]:.3f}')
  corr=np.corrcoef(squareform(confusion[25:50,25:50]),squareform(confusion[75:100,75:100]))
  print(f'Correlation between identity discrimination in upright and inverted unfamiliar faces is {corr[0,1]:.3f}')
  conf['perc'] = results[0]
  file_name='conf_mat_perc_' + str(i).zfill(2) + '.csv'
  fname = op.join(outfolder,file_name)
  np.savetxt(fname, conf['perc'], delimiter=',')